In [ ]:
# survived    标签
# sex
# age
# n_siblings_spouses
# parch
# fare
# class
# deck
# embark_town
# alone


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [4]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("d:\\Users\\Administrator\\Desktop\\train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("d:\\Users\\Administrator\\Desktop\\eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 5us/step


In [5]:
np.set_printoptions(precision=3, suppress= True)
# 设置打印操作
# precision  int or None  浮点数输出的精度位数
# suppress  bool 如果为真，则始终使用定点表示法打印浮点数，在这种情况下，当前精度为零的数字将打印为零。
#  如果为假，则当最小数的绝对值< 1e-4或最大绝对值与最小值之比为> 1e3时使用科学符号。
#  默认为False。

In [7]:
# 显示标识需要预测的列
LABEL_COLUMN = 'survived'
LABELS = [0,1]

In [137]:
def get_dataset(file_path, **kwargs):
    # Reads CSV files into a dataset.
    dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=5,
    label_name=LABEL_COLUMN,
    na_value="?",
    num_epochs= 1,
    ignore_errors=True, #忽略CSV文件解析中的错误
    **kwargs)
    return dataset

raw_train_data= get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)


In [138]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key,value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [139]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'female']
age                 : [ 6.  2. 28.  9.  4.]
n_siblings_spouses  : [0 1 0 4 1]
parch               : [1 1 0 2 1]
fare                : [12.475 26.     8.05  31.275 23.   ]
class               : [b'Third' b'Second' b'Third' b'Third' b'Second']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'n' b'y' b'n' b'n']


In [18]:
# 如果文件的第一行不包括列明
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses',
               'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [28. 24. 28. 23. 28.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 0.     7.496  7.775  7.896 79.2  ]
class               : [b'Second' b'Third' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [140]:
# 忽略数据中的某些列
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [48. 28. 43.  9. 35.]
n_siblings_spouses  : [1 8 1 5 1]
class               : [b'First' b'Third' b'Second' b'Third' b'First']
deck                : [b'D' b'unknown' b'unknown' b'unknown' b'C']
alone               : [b'n' b'n' b'n' b'n' b'n']


In [ ]:
# 数据预处理， 详见 Data preprocessing

In [141]:
# Continuous data
# 将适当格式的数据打包到向量中，传递给模型
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [26. 28. 34. 47. 19.]
n_siblings_spouses  : [0. 0. 1. 0. 0.]
parch               : [0. 0. 0. 0. 0.]
fare                : [ 7.775  0.    21.     9.    26.   ]


In [47]:
example_batch, labels_batch = next(iter(temp_dataset))

In [49]:
# 打包所有的列
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label
# 拼接后 [features label]

In [142]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 36.      1.      2.     27.75 ]
 [ 36.      1.      0.     26.   ]
 [ 28.      0.      0.    221.779]
 [ 47.      0.      0.      7.25 ]
 [ 34.      1.      0.     21.   ]]

[0 1 0 0 0]


In [51]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [28. 25. 80. 28.  4.]
n_siblings_spouses  : [1 1 0 0 3]
parch               : [0 0 0 0 2]
fare                : [ 7.75  17.8   30.     8.663 27.9  ]
class               : [b'Third' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'A' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'n']


In [89]:
# 更通用的预处理程序，它可以选择一个数字特性列表，并将它们打包成一列:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    # tf.cast (x, dtype,name= None)  将x 转为 dtype类型
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [90]:
L = tf.ones([3,4])
l = tf.stack(L,axis=-1)
print(L)
print(l)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(3, 4), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(4, 3), dtype=float32)


In [144]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))


packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [145]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'First' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'n' b'n' b'y']
numeric             : [[18.     1.     0.     6.496]
 [28.     0.     0.     7.796]
 [28.     0.     2.    15.246]
 [27.     1.     0.    53.1  ]
 [35.     0.     0.    26.288]]


In [93]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [146]:
# Data Normalization
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [107]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [109]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [147]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001B200753400>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [148]:
example_batch['numeric']

<tf.Tensor: id=1771, shape=(5, 4), dtype=float32, numpy=
array([[20.   ,  0.   ,  0.   ,  7.05 ],
       [28.   ,  0.   ,  0.   , 47.1  ],
       [18.   ,  1.   ,  0.   ,  6.496],
       [19.   ,  0.   ,  0.   ,  7.65 ],
       [28.   ,  0.   ,  0.   , 31.   ]], dtype=float32)>

In [149]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.77 , -0.474, -0.479, -0.501],
       [-0.13 , -0.474, -0.479,  0.233],
       [-0.93 ,  0.395, -0.479, -0.511],
       [-0.85 , -0.474, -0.479, -0.49 ],
       [-0.13 , -0.474, -0.479, -0.062]], dtype=float32)

In [112]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [156]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    # tf.feature_column.categorical_column_with_vocabulary_list
    # 当输入为字符串或整数格式，并且内存中的词汇表将每个值映射为整数ID时，请使用此选项。
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [157]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [158]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [ ]:
# Combined preprocessing layer

In [116]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [117]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.77  -0.474
 -0.479 -0.501  1.     0.   ]


In [ ]:
#  Build the model

In [118]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [119]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [120]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 4s 34ms/step - loss: 0.5065 - accuracy: 0.7815
Epoch 2/20
126/126 [==============================] - 1s 4ms/step - loss: 0.4244 - accuracy: 0.8262
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3990 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3938 - accuracy: 0.8309
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3909 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3702 - accuracy: 0.8341
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3608 - accuracy: 0.8389
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3584 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3517 - accuracy: 0.8517
Epoch 10/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3576 - accuracy: 0.858

In [121]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 2s 44ms/step - loss: 0.4579 - accuracy: 0.8295


Test Loss 0.45788990115782, Test Accuracy 0.8295454382896423


In [123]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 13.53%  | Actual outcome:  SURVIVED
Predicted survival: 81.96%  | Actual outcome:  DIED
Predicted survival: 14.71%  | Actual outcome:  DIED
Predicted survival: 87.81%  | Actual outcome:  DIED
Predicted survival: 14.07%  | Actual outcome:  DIED
